In [1]:
!ls

config	     pvc.yaml	 pv.yaml		     torchserve-pv.yaml
model-store  pvpod.yaml  torchserve-predictor.ipynb  ts.yaml


In [137]:
!kubectl apply -f pv.yaml -n kfserving-test

persistentvolume/model-pv-volume unchanged


In [138]:
!kubectl apply -f pvc.yaml -n kfserving-test

persistentvolumeclaim/model-pv-claim unchanged


In [139]:
!kubectl apply -f pvpod.yaml -n kfserving-test

pod/model-store-pod created


In [2]:
!kubectl exec -t model-store-pod -c model-store -n kfserving-test -- mkdir /pv/model-store/
!kubectl exec -t model-store-pod -c model-store -n kfserving-test -- mkdir /pv/config/

mkdir: cannot create directory '/pv/model-store/': File exists
command terminated with exit code 1
mkdir: cannot create directory '/pv/config/': File exists
command terminated with exit code 1


In [3]:
!kubectl cp model-store model-store-pod:/pv/ -c model-store -n kfserving-test

In [7]:
!kubectl cp config/config.properties model-store-pod:/pv/config/config.properties -c model-store -n kfserving-test

In [8]:
!kubectl exec -it model-store-pod -n kfserving-test -- ls -lR /pv/model-store/

/pv/model-store/:
total 8864
-rw-rw-r-- 1 1000 1000 4463882 Sep 14 15:01 mnist.mar
-rw-rw-r-- 1 1000 1000 4609382 Sep 14 15:01 squeezenet1_1.mar


In [9]:
!kubectl exec -it model-store-pod -n kfserving-test -- cat /pv/config/config.properties

inference_address=http://0.0.0.0:8080
management_address=http://0.0.0.0:8081
NUM_WORKERS=1
number_of_gpu=1
number_of_netty_threads=32
job_queue_size=1000
model_store=/home/model-server/shared/model-store
model_snapshot={"name":"startup.cfg","modelCount":2,"models":{"squeezenet1_1":{"1.0":{"defaultVersion":true,"marName":"squeezenet1_1.mar","minWorkers":3,"maxWorkers":3,"batchSize":1,"maxBatchDelay":100,"responseTimeout":120}},"mnist":{"1.0":{"defaultVersion":true,"marName":"mnist.mar","minWorkers":5,"maxWorkers":5,"batchSize":1,"maxBatchDelay":200,"responseTimeout":60}}}}


In [10]:
!kubectl delete -f pvpod.yaml -n kfserving-test

pod "model-store-pod" deleted


In [11]:
!cat torchserve-pv.yaml

apiVersion: serving.kubeflow.org/v1alpha2
kind: InferenceService
metadata:
  labels:
    controller-tools.k8s.io: "1.0"
  name: ts-sample
spec:
  default:
    predictor:
      torchserve:
        storageUri: "pvc://model-pv-claim"
        resources:
          limits:
            nvidia.com/gpu: 1

In [13]:
!kubectl apply -f torchserve-pv.yaml -n kfserving-test 

inferenceservice.serving.kubeflow.org/ts-sample created


In [14]:
!kubectl delete -f torchserve-pv.yaml -n kfserving-test

inferenceservice.serving.kubeflow.org "ts-sample" deleted
